<a href="https://colab.research.google.com/github/Eom-TaeJun/TJ/blob/main/20200573final_(2)_(2)_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 셀 1: 초기 설정 및 라이브러리 설치
%%capture
!pip install folium selenium webdriver_manager pandas numpy requests pillow opencv-python xgboost scikit-learn pyproj geopy matplotlib joblib tqdm beautifulsoup4 lxml flask
!pip install selenium pillow opencv-python pandas numpy folium



In [ ]:
#셀2
import os
import time
import json
import requests
import pandas as pd
import numpy as np
import folium
from pyproj import CRS, Transformer
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import random
from PIL import Image
import logging
import cv2
from pathlib import Path
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from geopy.distance import geodesic
from google.colab import drive
from concurrent.futures import ThreadPoolExecutor
import glob
# Google Drive 마운트
drive.mount('/content/drive')

 #기본 설정
BASE_PATH = '/content/drive/MyDrive/fanalysis'
CONFIG = {
    'API_KEY': 'F241130499',
    'BASE_URL': "http://www.opinet.co.kr/api/lowTop10.do",
    'IMAGE_PATH': f"{BASE_PATH}/images",
    'DATA_PATH': f"{BASE_PATH}/data",
    'SAMPLE_SIZE': 50
}
# 디렉토리 생성 함수
def setup_directories():
    """필요한 디렉토리 구조 생성"""
    directories = [
        BASE_PATH,
        CONFIG['DATA_PATH'],
        CONFIG['IMAGE_PATH'],
        f"{BASE_PATH}/logs"
    ]
    for directory in directories:
        os.makedirs(directory, exist_ok=True)
        print(f"디렉토리 확인/생성됨: {directory}")
def clear_cache():
    """캐시 파일 삭제"""
    cache_file = f"{BASE_PATH}/data/processed_images.pkl"
    if os.path.exists(cache_file):
        os.remove(cache_file)
        print(f"캐시 파일 삭제 완료: {cache_file}")
    else:
        print(f"캐시 파일이 없습니다: {cache_file}")
clear_cache()
def initialize_system():
    """시스템 초기화 및 설정"""
    print("시스템 초기화 중...")
    setup_directories()

    # API 연결 확인
    if not test_api_connection():
        raise Exception("API 연결 실패")

    # 캐시 정리
    clear_cache()
    print("시스템 초기화 완료.")
# 로깅 설정
logging.basicConfig(
    filename=f"{BASE_PATH}/analysis.log",
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
# 폴더 생성 및 경로 확인
print(f"현재 설정된 BASE_PATH: {BASE_PATH}")
print("\n폴더 생성 및 확인중...")

# 필요한 하위 폴더들 생성
os.makedirs(f"{BASE_PATH}/images", exist_ok=True)
os.makedirs(f"{BASE_PATH}/data", exist_ok=True)

# 생성된 폴더 확인
print("\n생성된 폴더 목록:")
!ls "/content/drive/MyDrive/fanalysis"




Mounted at /content/drive
캐시 파일이 없습니다: /content/drive/MyDrive/fanalysis/data/processed_images.pkl
현재 설정된 BASE_PATH: /content/drive/MyDrive/fanalysis

폴더 생성 및 확인중...

생성된 폴더 목록:
data  images


In [ ]:
# 셀 3: API 호출 및 데이터 수집 함수
def api_call_with_retry(url, params, max_retries=3, timeout=10):
    """API 호출 재시도 메커니즘"""
    for attempt in range(max_retries):
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0',
                'Accept': 'application/json'
            }
            print(f"API 호출 시도 {attempt + 1}/{max_retries}...")

            response = requests.get(url, params=params, headers=headers, timeout=timeout)
            print(f"응답 상태 코드: {response.status_code}")

            if response.status_code == 200:
                # HTML 응답 체크
                if '<html' in response.text or '<!DOCTYPE' in response.text:
                    print("HTML 응답 수신, 재시도...")
                    time.sleep(2)
                    continue

                print("API 응답 데이터 일부:")
                print(response.text[:200])  # 응답 데이터 일부 출력
                return response.json()
            else:
                print(f"API 호출 실패: {response.status_code} - {response.text}")
        except Exception as e:
            print(f"API 호출 오류: {e}")

        time.sleep(2 ** attempt)

    print("API 호출 실패. None 반환")
    return None

def validate_fetched_data(df):
    """수집된 데이터 유효성 검증"""
    if df is None or df.empty:
        print("수집된 데이터가 없습니다.")
        return False

    # NaN 값을 0으로 대체
    df.fillna(0, inplace=True)

    print(f"수집된 데이터 크기: {df.shape}")
    for col in ['PRICE', 'POLL_DIV_CD', 'OS_NM']:
        missing_values = df[col].isna().sum()
        print(f"{col} 열에서 NaN 값 수: {missing_values}")
    return True

def test_api_connection():
    """API 연결 테스트"""
    test_params = {
        'code': CONFIG['API_KEY'],  # API 키
        'out': 'json',              # 응답 형식 (JSON)
        'area': '01',               # 서울 지역 코드
        'prodcd': 'B027'            # 휘발유 코드
    }

    try:
        print("API 연결 테스트 중...")
        # BASE_URL 사용
        response = requests.get(CONFIG['BASE_URL'], params=test_params, timeout=10)

        # 응답 상태 코드 확인
        print(f"응답 상태 코드: {response.status_code}")

        if response.status_code == 200:
            # 응답 데이터 확인
            print("API 응답 데이터 일부:")
            print(json.dumps(response.json(), indent=4, ensure_ascii=False))

            # 예상된 구조 확인
            data = response.json()
            if 'RESULT' in data and 'OIL' in data['RESULT']:
                print("API 연결 테스트 성공: 데이터 구조 확인 완료")
                return True
            else:
                print("API 응답에 'RESULT' 또는 'OIL' 키가 없습니다.")
        else:
            print(f"API 호출 실패: 상태 코드 {response.status_code}")
    except Exception as e:
        print(f"API 연결 중 오류 발생: {e}")
    return False

def is_economical_station(station):
    """주유소가 알뜰주유소인지 확인"""
    try:
        poll_div_cd = station.get('POLL_DIV_CD', '').upper()
        van_co_cd = station.get('VAN_CO_CD', '').upper()
        os_nm = station.get('OS_NM', '').upper()

        eco_codes = ['SKE', 'GSC', 'HDO', 'SOL', 'RTO', 'RTX', 'NHO', 'NH', 'SK', 'GS', 'HD', 'SO', 'RT']
        eco_keywords = ['알뜰', '농협', 'NH']

        return any(code in [poll_div_cd, van_co_cd] for code in eco_codes) or any(keyword in os_nm for keyword in eco_keywords)

    except Exception as e:
        logging.error(f"알뜰주유소 확인 중 오류: {str(e)}")
        return False

def fetch_gas_station_data(config, force_update=False, limit=100):
    """수도권 주유소 데이터 수집"""
    print("데이터 수집 함수 호출...")
    cache_file = f"{config['DATA_PATH']}/stations_data.pkl"

    if os.path.exists(cache_file) and not force_update:
        print(f"캐시된 데이터를 불러옵니다: {cache_file}")
        df = pd.read_pickle(cache_file)
        print(f"캐시 데이터 크기: {df.shape}")
        return df.head(limit)  # 테스트용으로 일부 데이터만 반환

    try:
        print("API 호출을 시작합니다...")
        all_stations = []
        product_codes = ['B027']  # 휘발유만
        area_codes = {
            "01": "서울", "02": "경기", "03": "인천", "04": "강원", "05": "충북", "06": "충남",
            "07": "전북", "08": "전남", "09": "경북", "10": "경남", "11": "제주"
        }  # 전국 지역 코드

        for area_code, area_name in area_codes.items():
            for prodcd in product_codes:
                params = {
                    'code': config['API_KEY'],
                    'out': 'json',
                    'area': area_code,
                    'prodcd': prodcd
                }
                print(f"요청 파라미터: {params}")
                data = api_call_with_retry(config['BASE_URL'], params)
                if data and 'RESULT' in data and 'OIL' in data['RESULT']:
                    stations = data['RESULT']['OIL']
                    all_stations.extend(stations)

        df = pd.DataFrame(all_stations).drop_duplicates(subset=['UNI_ID'])
        print(f"최종 수집 데이터 크기: {df.shape}")
        df.to_pickle(cache_file)
        return df.head(limit)  # 테스트용 개수 제한 적용

    except Exception as e:
        print(f"데이터 수집 중 오류 발생: {e}")
        return None

# 실행 코드
if __name__ == "__main__":
    start_time = time.time()
    stations_df = fetch_gas_station_data(CONFIG)
    end_time = time.time()
    print(f"데이터 수집 완료. 수행 시간: {end_time - start_time:.4f}초")




데이터 수집 함수 호출...
API 호출을 시작합니다...
요청 파라미터: {'code': 'F241130499', 'out': 'json', 'area': '01', 'prodcd': 'B027'}
API 호출 시도 1/3...
응답 상태 코드: 200
API 응답 데이터 일부:

                                

 
{"RESULT":  
{"OIL":[ 
           
 
{"UNI_ID":"A0000427","PRICE":1595,"POLL_DIV_CD":"HDO","OS_NM":"영등포제일셀프주유소","VAN_ADR":"서울 영등포구 대림동 667-9","NEW_ADR":"서울
요청 파라미터: {'code': 'F241130499', 'out': 'json', 'area': '02', 'prodcd': 'B027'}
API 호출 시도 1/3...
응답 상태 코드: 200
API 응답 데이터 일부:

                                

 
{"RESULT":  
{"OIL":[ 
           
 
{"UNI_ID":"A0032941","PRICE":1589,"POLL_DIV_CD":"RTO","OS_NM":"(주)어울림주유소","VAN_ADR":"경기 고양시 덕양구 화정동 188-20","NEW_ADR":
요청 파라미터: {'code': 'F241130499', 'out': 'json', 'area': '03', 'prodcd': 'B027'}
API 호출 시도 1/3...
응답 상태 코드: 200
API 응답 데이터 일부:

                                

 
{"RESULT":  
{"OIL":[ 
           
 
{"UNI_ID":"A0010847","PRICE":1608,"POLL_DIV_CD":"HDO","OS_NM":"무궁화주유소","VAN_ADR":"강원 홍천군 홍천읍 하오안리 166-27","NEW_ADR":"강
요청 파라미터: {'co

In [ ]:
#셀4
def convert_utm_to_wgs84(utm_x, utm_y):
    """UTM 좌표를 WGS84 위경도로 변환"""
    try:
        print(f"좌표 변환 시도: UTM_X={utm_x}, UTM_Y={utm_y}")
        katec_proj = CRS.from_proj4(
            "+proj=tmerc +lat_0=38 +lon_0=128 +k=0.9999 +x_0=400000 +y_0=600000 +ellps=bessel "
            "+units=m +no_defs +towgs84=-115.80,474.99,674.11,1.16,-2.31,-1.63,6.43"
        )
        wgs84_proj = CRS.from_epsg(4326)
        transformer = Transformer.from_crs(katec_proj, wgs84_proj, always_xy=True)
        lon, lat = transformer.transform(float(utm_x), float(utm_y))
        print(f"변환된 좌표: LAT={lat}, LON={lon}")
        return lat, lon
    except Exception as e:
        print(f"좌표 변환 오류: {e}")
        return None, None



def create_map(lat, lon):
    """지도 생성 및 저장"""
    try:
        # 마커 없이 지도 생성
        map_ = folium.Map(location=[lat, lon], zoom_start=15, width=800, height=800)
        return map_
    except Exception as e:
        print(f"지도 생성 오류: {e}")
        return None


def get_station_prices(api_key, rad_lat, rad_lon, radius):
    """특정 반경 내의 주유소 가격 정보 조회"""
    try:
        katec_proj = CRS.from_proj4(
            "+proj=tmerc +lat_0=38 +lon_0=128 +k=0.9999 +x_0=400000 +y_0=600000 +ellps=bessel "
            "+units=m +no_defs +towgs84=-115.80,474.99,674.11,1.16,-2.31,-1.63,6.43"
        )
        wgs84_proj = CRS.from_epsg(4326)
        transformer = Transformer.from_crs(wgs84_proj, katec_proj, always_xy=True)
        x_katec, y_katec = transformer.transform(rad_lon, rad_lat)

        url = "http://www.opinet.co.kr/api/aroundAll.do"
        params = {
            'code': api_key,
            'out': 'json',
            'x': str(int(x_katec)),
            'y': str(int(y_katec)),
            'radius': str(int(radius * 1000)),
            'prodcd': 'B027'
        }

        data = api_call_with_retry(url, params)
        if data and 'RESULT' in data and 'OIL' in data['RESULT']:
            return data['RESULT']['OIL']
        return []

    except Exception as e:
        logging.error(f"가격 정보 조회 중 오류: {str(e)}")
        return []

def analyze_price_impact(station, config):
    """주유소 주변 가격 영향 분석"""
    try:
        utm_x = float(station["GIS_X_COOR"])
        utm_y = float(station["GIS_Y_COOR"])
        lat, lon = convert_utm_to_wgs84(utm_x, utm_y)

        print(f"\n{station['OS_NM']} 주변 가격 분석 중...")

        inner_radius = 1.6
        outer_radius = 3.2

        inner_stations = get_station_prices(config['API_KEY'], lat, lon, inner_radius)
        time.sleep(1)  # API 요청 간격 조절
        outer_stations = get_station_prices(config['API_KEY'], lat, lon, outer_radius)

        inner_prices = []
        outer_prices = []

        for s in inner_stations:
            if s.get('PRICE'):
                try:
                    price = float(s['PRICE'])
                    if price > 0:
                        inner_prices.append(price)
                except ValueError:
                    logging.warning(f"Invalid price format in inner stations: {s['PRICE']}")

        for s in outer_stations:
            try:
                s_lat, s_lon = convert_utm_to_wgs84(float(s["GIS_X_COOR"]), float(s["GIS_Y_COOR"]))
                distance = geodesic((lat, lon), (s_lat, s_lon)).kilometers
                if inner_radius < distance <= outer_radius and s.get('PRICE'):
                    price = float(s['PRICE'])
                    if price > 0:
                        outer_prices.append(price)
            except (ValueError, KeyError):
                logging.warning(f"Invalid data in outer stations: {s}")

        if not (inner_prices or outer_prices):
            print("가격 데이터가 없습니다")
            return None

        result = {
            'inner_station_count': len(inner_prices),
            'outer_station_count': len(outer_prices),
            'inner_avg_price': round(sum(inner_prices) / len(inner_prices), 2) if inner_prices else 0,
            'outer_avg_price': round(sum(outer_prices) / len(outer_prices), 2) if outer_prices else 0,
            'price_impact': round((sum(outer_prices) / len(outer_prices) if outer_prices else 0) -
                                (sum(inner_prices) / len(inner_prices) if inner_prices else 0), 2)
        }

        print(f"분석 결과: {result}")
        return result

    except Exception as e:
        logging.error(f"{station['OS_NM']} 분석 중 오류: {str(e)}")
        return None


In [ ]:
#셀5
import os
import re

def save_map_for_cnn(lat, lon, station_name, config):
    """지도 HTML을 생성하고 이미지로 저장"""
    try:
        # 안전한 이름 전처리
        safe_name = ''.join(c for c in station_name if c.isalnum())

        # HTML 및 이미지 경로 설정
        html_path = os.path.join(config['IMAGE_PATH'], f'{safe_name}.html')
        final_image_path = os.path.join(config['IMAGE_PATH'], f'{safe_name}.jpg')  # 접두사/접미사 제거

        # 지도 생성 및 저장 (마커 없음)
        map_ = folium.Map(location=[lat, lon], zoom_start=15, width=800, height=800)
        map_.save(html_path)

        # HTML 파일을 이미지로 변환
        return save_html_as_image(html_path, final_image_path)

    except Exception as e:
        print(f"지도 생성 중 오류: {e}")
        return None

def save_html_as_image(html_path, final_image_path):
    """HTML 파일을 이미지로 변환 (여백 제거 포함)"""
    driver = None
    try:
        print(f"HTML 파일을 이미지로 변환 중: {html_path}")

        # Selenium 드라이버 설정
        options = webdriver.ChromeOptions()
        options.add_argument('--headless=new')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--window-size=800,800')  # 윈도우 크기 설정
        options.add_argument('--disable-gpu')
        options.add_argument('--hide-scrollbars')

        driver = webdriver.Chrome(options=options)
        driver.get(f'file:///{os.path.abspath(html_path)}')
        time.sleep(2)  # 로딩 대기

        # 스크린샷 캡처
        temp_path = final_image_path.replace('.jpg', '_temp.png')
        driver.save_screenshot(temp_path)

        # 이미지를 열어 여백 제거
        image = Image.open(temp_path)
        cropped_image = image.crop((0, 0, image.width, image.height))  # 여백 제거
        cropped_image.save(final_image_path, 'JPEG', quality=95)

        print(f"이미지 저장 완료: {final_image_path}")
        os.remove(temp_path)
        os.remove(html_path)  # HTML 파일 삭제

        return final_image_path
    except Exception as e:
        print(f"이미지 변환 중 오류: {e}")
        return None
    finally:
        if driver:
            driver.quit()




def extract_image_features(image_path):
    """이미지에서 특성 추출"""
    try:
        img = cv2.imread(image_path)
        if img is None:
            return None

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        features = {
            'mean_intensity': np.mean(gray),
            'std_intensity': np.std(gray),
            'median_intensity': np.median(gray),
            'road_density': np.sum(gray > 200) / gray.size,
            'building_density': np.sum((gray > 100) & (gray < 200)) / gray.size
        }

        edges = cv2.Canny(gray, 100, 200)
        features['edge_density'] = np.sum(edges > 0) / edges.size

        h, w = gray.shape
        quadrants = [
            gray[:h//2, :w//2],
            gray[:h//2, w//2:],
            gray[h//2:, :w//2],
            gray[h//2:, w//2:]
        ]

        for i, quad in enumerate(quadrants):
            features[f'quadrant_{i+1}_density'] = np.mean(quad)

        return features

    except Exception as e:
        logging.error(f"이미지 특성 추출 중 오류: {str(e)}")
        return None

def process_images(stations_df, config, limit=100):
    """주유소 이미지 처리 및 price_impact 추가"""
    print("\n이미지 처리 프로세스 시작...")

    processed_data = []
    for idx, (_, station) in enumerate(stations_df.head(limit).iterrows(), 1):
        try:
            # 좌표 변환
            lat, lon = convert_utm_to_wgs84(station["GIS_X_COOR"], station["GIS_Y_COOR"])
            if lat is None or lon is None:
                print(f"좌표 변환 실패: {station['OS_NM']}")
                continue

            # 지도 이미지 저장
            image_path = save_map_for_cnn(lat, lon, station["OS_NM"], config)
            if image_path:
                print(f"지도 이미지 저장 성공: {image_path}")
            else:
                print(f"지도 이미지 저장 실패: {station['OS_NM']}")

            # 가격 영향 분석
            price_impact_result = analyze_price_impact(station, config)
            price_impact = price_impact_result['price_impact'] if price_impact_result else 0

            # 데이터 병합
            station_data = {
                'station_name': station['OS_NM'],
                'latitude': lat,
                'longitude': lon,
                'image_path': image_path,
                'price_impact': price_impact,  # price_impact 추가
                **station.to_dict()
            }
            processed_data.append(station_data)
        except Exception as e:
            print(f"데이터 처리 오류: {e}")
            continue

    if not processed_data:
        raise ValueError("처리된 데이터가 없습니다.")

    result_df = pd.DataFrame(processed_data)
    result_df.to_pickle(f"{config['DATA_PATH']}/processed_images.pkl")
    print("최종 데이터 저장 완료.")
    return result_df






In [ ]:
#셀6
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

def preprocess_data(df, target_column='price_impact'):
    """데이터 전처리: 특성(X)과 타겟(y) 분리"""
    features = [col for col in df.columns if col != target_column]  # 타겟 열 제외
    X = df[features]  # 입력 데이터
    y = df[target_column]  # 타겟 값
    return X, y
    print("처리된 데이터 샘플:", processed_df.head())
    if 'price_impact' not in processed_df.columns:
      raise ValueError("`price_impact` 열이 처리된 데이터에 없습니다.")

def train_model(X, y):
    """모델 학습"""
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)
    model.fit(X_train, y_train)

    return model, X_test, y_test



In [ ]:
#셀7
def main():
    """메인 실행 함수"""
    try:
        print("=== 주유소 데이터 분석 시스템 시작 ===")

        # 1. 초기 설정 및 시스템 초기화
        initialize_system()

        # 2. 데이터 수집
        stations_df = fetch_gas_station_data(CONFIG)
        if stations_df is None or stations_df.empty:
            raise Exception("주유소 데이터 수집 실패")
        print(f"수집된 주유소 데이터: {len(stations_df)}개")

        # 3. 데이터 처리 (지도 이미지 생성 포함)
        processed_df = process_images(stations_df, CONFIG)
        if processed_df is None or processed_df.empty:
            raise Exception("데이터 처리 결과가 없습니다")

        # 4. 데이터 전처리 및 모델 학습
        X, y = preprocess_data(processed_df, target_column='price_impact')
        model, X_test, y_test = train_model(X, y)

        # 5. 결과 저장
        timestamp = time.strftime('%Y%m%d_%H%M%S')
        results_path = f"{CONFIG['DATA_PATH']}/analysis_results_{timestamp}"
        os.makedirs(results_path, exist_ok=True)

        processed_df.to_csv(f"{results_path}/processed_data.csv", index=False)
        print("\n=== 분석 완료 ===")
        print(f"결과가 저장된 경로: {results_path}")

    except Exception as e:
        logging.error(f"분석 중 오류 발생: {str(e)}")
        print(f"오류 발생: {str(e)}")


if __name__ == "__main__":
    try:
        print("=== 시스템 초기화 ===")
        initialize_system()

        # 데이터 수집 및 처리
        test_limit = 100  # 테스트 시 출력할 주유소 개수
        stations_df = fetch_gas_station_data(CONFIG, limit=test_limit)
        print(f"수집된 데이터 크기: {stations_df.shape}")

        processed_df = process_images(stations_df, CONFIG, limit=test_limit)
        print("데이터 처리 완료")
    except Exception as e:
        print(f"오류 발생: {e}")


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
좌표 변환 시도: UTM_X=364019.7578, UTM_Y=410895.0442
변환된 좌표: LAT=36.29797315589265, LON=127.5972176693751
HTML 파일을 이미지로 변환 중: /content/drive/MyDrive/analysis/images/보령물산주옥천서울주유소충전소.html
이미지 저장 완료: /content/drive/MyDrive/analysis/images/보령물산주옥천서울주유소충전소.jpg
지도 이미지 저장 성공: /content/drive/MyDrive/analysis/images/보령물산주옥천서울주유소충전소.jpg
좌표 변환 시도: UTM_X=364019.7578, UTM_Y=410895.0442
변환된 좌표: LAT=36.29797315589265, LON=127.5972176693751

보령물산(주)옥천(서울)주유소/충전소 주변 가격 분석 중...
API 호출 시도 1/3...
응답 상태 코드: 200
API 응답 데이터 일부:

                                

 


{"RESULT":  

{"OIL":[ 
 
                                          

{"UNI_ID":"A0012713","POLL_DIV_CD":"RTX","OS_NM":"보령물산(주)옥천(서울)주유소/충전소","PRICE"
API 호출 시도 1/3...
응답 상태 코드: 200
API 응답 데이터 일부:

                                

 


{"RESULT":  

{"OIL":[ 
 
                                          

{"UNI_ID":"A0012713","POLL_DIV_CD":"RTX","OS_NM":"보령물산(주)옥천(서울)주유소/충전소","PRICE"
좌표 변환 시도: UTM_X=364019.7578, UTM_Y=

In [ ]:
import pandas as pd

# 저장된 파일 불러오기
processed_data_path = f"{CONFIG['DATA_PATH']}/processed_images.pkl"
processed_df = pd.read_pickle(processed_data_path)

# 데이터 확인
print("처리된 데이터 샘플:")
print(processed_df.head())

# 데이터 구조 확인
print("\n데이터프레임 정보:")
print(processed_df.info())

# price_impact 열 확인
if 'price_impact' in processed_df.columns:
    print("\n'price_impact' 열 존재 확인:")
    print(processed_df['price_impact'].describe())
else:
    print("경고: 'price_impact' 열이 존재하지 않습니다.")

처리된 데이터 샘플:
             station_name   latitude   longitude  \
0              영등포제일셀프주유소  37.502275  126.899546   
1                남서울고속주유소  37.501542  126.898901   
2               서울중앙고속주유소  37.561843  127.074794   
3         (주)타이거통상 월드컵주유소  37.567508  126.909693   
4  (주)아이에스피에너지(서울서부고속주유소)  37.586740  126.915969   

                                          image_path  price_impact    UNI_ID  \
0  /content/drive/MyDrive/analysis/images/영등포제일셀프...         57.27  A0000427   
1  /content/drive/MyDrive/analysis/images/남서울고속주유...         56.53  A0000412   
2  /content/drive/MyDrive/analysis/images/서울중앙고속주...          7.12  A0009330   
3  /content/drive/MyDrive/analysis/images/주타이거통상월...         28.81  A0000139   
4  /content/drive/MyDrive/analysis/images/주아이에스피에...         32.17  A0005900   

   PRICE POLL_DIV_CD                   OS_NM              VAN_ADR  \
0   1595         HDO              영등포제일셀프주유소    서울 영등포구 대림동 667-9   
1   1595         SKE                남서울고속주유소  서울 영등포구 대림

In [ ]:
import os

# 파일 경로 설정
file_path = f"{CONFIG['DATA_PATH']}/processed_images.pkl"

# 파일 존재 여부 확인
if os.path.exists(file_path):
    print(f"파일이 존재합니다: {file_path}")
else:
    print(f"파일이 존재하지 않습니다: {file_path}")

파일이 존재합니다: /content/drive/MyDrive/analysis/data/processed_images.pkl


In [ ]:
import os

image_dir = CONFIG['IMAGE_PATH']

if os.path.exists(image_dir):
    print(f"이미지 폴더 경로: {image_dir}")
    print("폴더 내용:")
    files = os.listdir(image_dir)
    print(files if files else "폴더가 비어 있습니다.")
else:
    print("이미지 폴더가 생성되지 않았습니다.")

이미지 폴더 경로: /content/drive/MyDrive/analysis/images
폴더 내용:
['영등포제일셀프주유소.jpg', '남서울고속주유소.jpg', '서울중앙고속주유소.jpg', '주타이거통상월드컵주유소.jpg', '주아이에스피에너지서울서부고속주유소.jpg', 'HD현대오일뱅크직영남부순환셀프주유소.jpg', '주디오티디은평유니콘주유소.jpg', '한화미구파발주유소.jpg', '타이거주유소.jpg', '주대청에너지대청주유소.jpg', '주어울림주유소.jpg', '주디오티디용인드래곤주유소.jpg', 'SKKH에너지가나주유소.jpg', '주원흥풍동지점.jpg', '덕양제7주유소.jpg', '덕양제1주유소.jpg', '일광에너비스덕이동점신흥에너비스.jpg', '주디오티디고양야옹이주유소.jpg', '너멍골.jpg', 'KH에너지주직영애니골2주유소.jpg', '회산주유소.jpg', '문막인천방향주유소.jpg', '무궁화주유소.jpg', '솔향주유소.jpg', '주선인상사홍천강춘천방향주유소충전소.jpg', '강릉주유소.jpg', '주대청산업개발원주춘천주유소.jpg', '구정속초주유소.jpg', '씨앤에스에너지주횡성하주유소충전소.jpg', '송정주유소.jpg', 

In [ ]:
import zipfile
import os

# 이미지 폴더 경로
image_dir = CONFIG['IMAGE_PATH']

# ZIP 파일 경로
zip_path = '/content/drive/MyDrive/fanalysis/images.zip'

# ZIP 파일 생성
with zipfile.ZipFile(zip_path, 'w') as zipf:
    for root, dirs, files in os.walk(image_dir):
        for file in files:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, image_dir)  # 상대 경로로 추가
            zipf.write(file_path, arcname)
print(f"ZIP 파일 생성 완료: {zip_path}")

ZIP 파일 생성 완료: /content/drive/MyDrive/analysis/images.zip
